# Statistical and Machine Learning Approaches for Marketing
## Homework: Neural Networks with Tensorflow and Karet
Fernando Delgado

In [317]:
wd = "C:/Users/fdelgado/OneDrive - IESEG/Documents/01. IESEG/13. Statistical and Machine Learning Approaches/SML_Section7"

In [318]:
# Check Python version
import sys
print('Python version:', sys.version)

# Some important functions and libraries
from __future__ import absolute_import, division, print_function, unicode_literals
import numpy as np
import pandas as pd
import random
import matplotlib.pyplot as plt

import tensorflow as tf
print('Tensorflow version:', tf.__version__)

#Machine Learning Models
from sklearn import linear_model
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import RandomForestRegressor


#Scores
import rfpimp
from sklearn import metrics
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score

Python version: 3.9.4 (default, Apr  9 2021, 11:43:21) [MSC v.1916 64 bit (AMD64)]
Tensorflow version: 2.8.0


In [319]:
#Train/Test
import sklearn
from sklearn.model_selection import train_test_split

Q1: Randomly divide data into train/test as 80/20 (set.seed =1).

In [320]:
df = pd.read_csv(wd + '/data/bankruptcy_prediction/data.csv')
df.head()

,Bankrupt?,ROA(C) before interest and depreciation before interest,ROA(A) before interest and % after tax,ROA(B) before interest and depreciation after tax,Operating Gross Margin,Realized Sales Gross Margin,Operating Profit Rate,Pre-tax net Interest Rate,After-tax net Interest Rate,Non-industry income and expenditure/revenue,...,Net Income to Total Assets,Total assets to GNP price,No-credit Interval,Gross Profit to Sales,Net Income to Stockholder's Equity,Liability to Equity,Degree of Financial Leverage (DFL),Interest Coverage Ratio (Interest expense to EBIT),Net Income Flag,Equity to Liability
0,1,0.370594,0.424389,0.405750,0.601457,0.601457,0.998969,0.796887,0.808809,0.302646,...,0.716845,0.009219,0.622879,0.601453,0.827890,0.290202,0.026601,0.564050,1,0.016469
1,1,0.464291,0.538214,0.516730,0.610235,0.610235,0.998946,0.797380,0.809301,0.303556,...,0.795297,0.008323,0.623652,0.610237,0.839969,0.283846,0.264577,0.570175,1,0.020794
2,1,0.426071,0.499019,0.472295,0.601450,0.601364,0.998857,0.796403,0.808388,0.302035,...,0.774670,0.040003,0.623841,0.601449,0.836774,0.290189,0.026555,0.563706,1,0.016474
3,1,0.399844,0.451265,0.457733,0.583541,0.583541,0.998700,0.796967,0.808966,0.303350,...,0.739555,0.003252,0.622929,0.583538,0.834697,0.281721,0.026697,0.564663,1,0.023982
4,1,0.465022,0.538432,0.522298,0.598783,0.598783,0.998973,0.797366,0.809304,0.303475,...,0.795016,0.003878,0.623521,0.598782,0.839973,0.278514,0.024752,0.575617,1,0.035490


In [321]:
predictors = df.loc[:, df.columns != 'Bankrupt?']
target = df['Bankrupt?']

In [322]:
X_train, X_test, y_train, y_test = train_test_split(predictors, target, test_size = 0.2)

In [323]:
print("Train shape: ", X_train.shape)
print("Test shape: ", X_test.shape)

Train shape:  (5455, 95)
Test shape:  (1364, 95)


Q2: Build a NN model with 1 hidden layer of 30 neurons, sigmoid activation function.

In [324]:
# Simple NN model
# Note: this model is untrained
model = tf.keras.models.Sequential([
  tf.keras.layers.Dense(30, input_dim=95, activation='sigmoid'), # Hidden layer
  tf.keras.layers.Dense(10)                                       # Output layer
])

In [325]:
model.summary()

Model: "sequential_30"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_78 (Dense)            (None, 30)                2880      
                                                                 
 dense_79 (Dense)            (None, 10)                310       
                                                                 
Total params: 3,190
Trainable params: 3,190
Non-trainable params: 0
_________________________________________________________________


In [326]:
#Some predictions
predictions = model(np.array(X_train[:1]))
predictions

<tf.Tensor: shape=(1, 10), dtype=float32, numpy=
array([[-0.59579796,  0.43705598,  1.8019608 ,  0.1950848 ,  0.58735025,
        -0.19092764,  0.22980948, -1.0820106 ,  0.97327715, -0.2729496 ]],
      dtype=float32)>

In [327]:
# Return prediction as a vector of probability
tf.nn.softmax(predictions).numpy()

array([[0.03240637, 0.0910318 , 0.35642138, 0.07146713, 0.10579499,
        0.04858065, 0.0739924 , 0.01992833, 0.155622  , 0.044755  ]],
      dtype=float32)

In [328]:
# Define the loss function (or error of the prediction)
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
loss_fn(y_train[:1], predictions).numpy()

3.4294004

In [329]:
# Select the optimize
# Adam: Adaptive moment estimation (Source: https://arxiv.org/pdf/1412.6980.pdf)
model.compile(optimizer='adam',
              loss=loss_fn,
              metrics=['accuracy'])

In [330]:
# Train the model
model.fit(X_train, y_train, epochs=5)

Epoch 1/5
171/171 [==============================] - 1s 2ms/step - loss: 1.4910 - accuracy: 0.5872
Epoch 2/5
171/171 [==============================] - 0s 2ms/step - loss: 0.3227 - accuracy: 0.9674
Epoch 3/5
171/171 [==============================] - 0s 2ms/step - loss: 0.2136 - accuracy: 0.9674
Epoch 4/5
171/171 [==============================] - 0s 2ms/step - loss: 0.1835 - accuracy: 0.9674
Epoch 5/5
171/171 [==============================] - 0s 2ms/step - loss: 0.1694 - accuracy: 0.9674


In [331]:
# Evaluate the model with the loss function and the accuracy
model.evaluate(X_train,  y_train, verbose=2)

171/171 - 0s - loss: 0.1643 - accuracy: 0.9674 - 378ms/epoch - 2ms/step


[0.16434510052204132, 0.9673693776130676]

In [332]:
# Evaluate the model with the loss function and the accuracy
model.evaluate(X_test,  y_test, verbose=2)

43/43 - 0s - loss: 0.1619 - accuracy: 0.9692 - 85ms/epoch - 2ms/step


[0.16193678975105286, 0.9692082405090332]

In [333]:
# Wrap the model by a softmax layer
probability_model = tf.keras.Sequential([
  model,
  tf.keras.layers.Softmax()
])

In [334]:
# Make prediction
y_test_pred = probability_model(np.array(X_test))
y_test_pred = y_test_pred.numpy()
y_test_pred[:5]

array([[9.7284085e-01, 2.3197034e-02, 1.3172361e-03, 5.2303821e-04,
        2.9435661e-04, 4.1722140e-04, 3.0482694e-04, 6.8338450e-05,
        7.0100802e-04, 3.3614400e-04],
       [9.5594287e-01, 2.7590133e-02, 6.1710952e-03, 8.6859724e-04,
        9.8778948e-04, 1.7648324e-03, 2.0623892e-03, 5.9582526e-04,
        2.2959709e-03, 1.7204862e-03],
       [9.6926731e-01, 2.7089028e-02, 1.7505703e-03, 2.5690679e-04,
        1.9454991e-04, 1.4714249e-04, 3.2649361e-04, 1.7381890e-04,
        5.8032514e-04, 2.1383679e-04],
       [9.8101556e-01, 1.5105475e-02, 1.2290512e-03, 9.5667178e-04,
        8.9400201e-05, 4.7065105e-04, 5.0074083e-04, 1.1719292e-04,
        3.0311293e-04, 2.1231826e-04],
       [9.1899627e-01, 4.4549510e-02, 9.7334953e-03, 4.2247628e-03,
        3.7706017e-03, 7.1230484e-03, 2.3249295e-03, 9.4022229e-04,
        3.7539839e-03, 4.5832456e-03]], dtype=float32)

Q3: Build a deep NN model with multiple hidden layers (of your choice) and sigmoid activation function.

In [335]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Dense(100, input_dim=95, activation='sigmoid'), # Hidden layer
  tf.keras.layers.Dense(50, activation='sigmoid'),
  tf.keras.layers.Dense(20, activation='sigmoid'),
  tf.keras.layers.Dense(5, activation='sigmoid'),
  tf.keras.layers.Dense(25, activation='sigmoid'),
  tf.keras.layers.Dense(6, activation='sigmoid'),
  tf.keras.layers.Dense(12, activation='sigmoid'),
  tf.keras.layers.Dense(10)                                       # Output layer
])

In [336]:
model.summary()

Model: "sequential_32"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_80 (Dense)            (None, 100)               9600      
                                                                 
 dense_81 (Dense)            (None, 50)                5050      
                                                                 
 dense_82 (Dense)            (None, 20)                1020      
                                                                 
 dense_83 (Dense)            (None, 5)                 105       
                                                                 
 dense_84 (Dense)            (None, 25)                150       
                                                                 
 dense_85 (Dense)            (None, 6)                 156       
                                                                 
 dense_86 (Dense)            (None, 12)              

In [337]:
#Some predictions
predictions = model(np.array(X_train))
predictions

<tf.Tensor: shape=(5455, 10), dtype=float32, numpy=
array([[ 1.327379  ,  0.07943282, -0.05306058, ..., -0.34971264,
        -0.3018614 , -0.01765831],
       [ 1.3274951 ,  0.07951418, -0.05304661, ..., -0.3496171 ,
        -0.30178034, -0.01763217],
       [ 1.3275397 ,  0.0795192 , -0.05308502, ..., -0.34967262,
        -0.30192173, -0.01761303],
       ...,
       [ 1.3275008 ,  0.07948819, -0.053011  , ..., -0.34968436,
        -0.30194756, -0.01760544],
       [ 1.3276172 ,  0.07947566, -0.05295801, ..., -0.349707  ,
        -0.30207053, -0.01755869],
       [ 1.3275071 ,  0.07944685, -0.05298862, ..., -0.34972316,
        -0.30200177, -0.01761325]], dtype=float32)>

In [338]:
# Return prediction as a vector of probability
tf.nn.softmax(predictions).numpy()

array([[0.27752474, 0.07967564, 0.06978859, ..., 0.05187404, 0.05441662,
        0.07230352],
       [0.27753899, 0.07967696, 0.06978504, ..., 0.05187563, 0.05441751,
        0.07230073],
       [0.27754965, 0.07967687, 0.06978194, ..., 0.05187244, 0.05440948,
        0.07230166],
       ...,
       [0.2775401 , 0.07967475, 0.06978741, ..., 0.05187205, 0.05440832,
        0.07230253],
       [0.27755505, 0.07966878, 0.06978674, ..., 0.05186764, 0.05439823,
        0.0723014 ],
       [0.2775402 , 0.079671  , 0.06978855, ..., 0.05186974, 0.05440504,
        0.07230154]], dtype=float32)

In [339]:
# Define the loss function (or error of the prediction)
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
loss_fn(y_train, predictions).numpy()

1.3225108

In [340]:
# Select the optimize
# Adam: Adaptive moment estimation (Source: https://arxiv.org/pdf/1412.6980.pdf)
model.compile(optimizer='adam',
              loss=loss_fn,
              metrics=['accuracy'])

In [341]:
# Train the model
model.fit(X_train, y_train, epochs=1000)

Epoch 1/1000
171/171 [==============================] - 1s 2ms/step - loss: 0.6267 - accuracy: 0.9674
Epoch 2/1000
171/171 [==============================] - 0s 2ms/step - loss: 0.2201 - accuracy: 0.9674
Epoch 3/1000
171/171 [==============================] - 0s 2ms/step - loss: 0.1744 - accuracy: 0.9674
Epoch 4/1000
171/171 [==============================] - 0s 2ms/step - loss: 0.1605 - accuracy: 0.9674
Epoch 5/1000
171/171 [==============================] - 0s 2ms/step - loss: 0.1545 - accuracy: 0.9674
Epoch 6/1000
171/171 [==============================] - 0s 2ms/step - loss: 0.1514 - accuracy: 0.9674
Epoch 7/1000
171/171 [==============================] - 0s 2ms/step - loss: 0.1494 - accuracy: 0.9674
Epoch 8/1000
171/171 [==============================] - 0s 2ms/step - loss: 0.1481 - accuracy: 0.9674
Epoch 9/1000
171/171 [==============================] - 0s 2ms/step - loss: 0.1473 - accuracy: 0.9674
Epoch 10/1000
171/171 [==============================] - 0s 2ms/step - loss: 0.146

In [342]:
# Evaluate the model with the loss function and the accuracy
result = model.evaluate(X_train,  y_train, verbose=2)

perf_train = pd.DataFrame({"Train_Accuracy":result[1]}, index = ['NeuralNetwork'])
perf_train

171/171 - 1s - loss: 0.0584 - accuracy: 0.9864 - 609ms/epoch - 4ms/step


,Train_Accuracy
NeuralNetwork,0.986434


In [343]:
# Evaluate the model with the loss function and the accuracy
result = model.evaluate(X_test,  y_test, verbose=2)

#set into df
perf_test = pd.DataFrame({"Test_Accuracy":result[1]}, index = ['NeuralNetwork'])
perf_test

43/43 - 0s - loss: 0.2257 - accuracy: 0.9465 - 107ms/epoch - 2ms/step


,Test_Accuracy
NeuralNetwork,0.946481


In [344]:
#Compare Train and Test AUC
perf_nn = perf_train.merge(perf_test, left_index=True, right_index=True)
perf_nn

,Train_Accuracy,Test_Accuracy
NeuralNetwork,0.986434,0.946481


Q4: Build Other 5 Classification models and compare

In [345]:
#Models

logistic     = LogisticRegression()
randomForest = RandomForestClassifier()
boostedTree  = GradientBoostingClassifier()
XG           = XGBClassifier()
KNN          = KNeighborsClassifier()

# create a dict to loop through the models later on
models = {
          "logistic"     :logistic,
          "randomForest" :randomForest,
          "boostedTree"  :boostedTree,
          "XG"           :XG,
          "KNN"          :KNN
         }

In [346]:
#Other 
import shutup; shutup.please()

In [347]:
# fit the models
#https://github.com/sagarnanduunc/Neural-Net-for-Default-of-Credit-Card-Clients/blob/master/Neural%20Net%20using%20TensorFlow.ipynb
#https://github.com/sagarnanduunc/Neural-Net-for-Default-of-Credit-Card-Clients

for model in models:
    random.seed=1
    models[model].fit(X_train, y_train.values.ravel())
    print(f"{model} has been trained successfully")

logistic has been trained successfully
randomForest has been trained successfully
boostedTree has been trained successfully
[18:04:18] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
XG has been trained successfully
KNN has been trained successfully


In [348]:
performances = {}

for model in models:
    predictions   = models[model].predict(X_train)
    accuracy      = accuracy_score(y_train, predictions)
    print(f"{model} completed.")
    performances[model] = {"Train_Accuracy":accuracy}

train_perf = pd.DataFrame(performances).transpose()

logistic completed.
randomForest completed.
boostedTree completed.
XG completed.
KNN completed.


In [349]:
performances = {}

for model in models:
    random.seed=1
    predictions   = models[model].predict(X_test)
    accuracy      = accuracy_score(y_test, predictions)
    print(f"{model} completed.")
    performances[model] = {"Test_Accuracy":accuracy}

test_perf = pd.DataFrame(performances).transpose()

logistic completed.
randomForest completed.
boostedTree completed.
XG completed.
KNN completed.


In [350]:
#Compare Train and Test AUC
perf_comp = train_perf.merge(test_perf, left_index=True, right_index=True)

In [351]:
perf_comp.append(perf_nn).sort_values(by="Test_Accuracy", ascending=False)

,Train_Accuracy,Test_Accuracy
randomForest,1.000000,0.969208
boostedTree,0.993951,0.968475
XG,1.000000,0.968475
KNN,0.969203,0.968475
logistic,0.960403,0.963343
NeuralNetwork,0.986434,0.946481
